In [ ]:
%pip install -U azure-ai-ml

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml.entities import MarketplaceSubscription, ServerlessEndpoint

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

client = MLClient(
    credential=credential,
    subscription_id="<subscription-id>",
    resource_group_name="<resource-group-name>",
    workspace_name="<workspace-name>",
)

In [ ]:
# Do this if needed, does not work currently
# but llama3 example works from here - https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-serverless?tabs=python

model_id="azureml://registries/azureml/models/Phi-3-mini-4k-instruct"
subscription_name="Phi-3-mini-4k-instruct"

marketplace_subscription = MarketplaceSubscription(
    model_id=model_id,
    name=subscription_name,
)

marketplace_subscription = client.marketplace_subscriptions.begin_create_or_update(
    marketplace_subscription
).result()


In [ ]:
# Deploy the model as a serverless endpoint
endpoint_name="phi3-mini-4k-instruct-qwerty2" # Name must be unique

model_id = "azureml://locations/eastus2/workspaces/bc8d2dc0-5d80-47af-bec7-566cfac2cd18/models/finetuned-model-86b380/versions/1"


In [ ]:
serverless_endpoint = ServerlessEndpoint(
    name=endpoint_name,
    model_id=model_id
)

created_endpoint = client.serverless_endpoints.begin_create_or_update(
    serverless_endpoint
).result()

In [ ]:
endpoint = client.serverless_endpoints.get(endpoint_name)

In [ ]:
import requests

url = f"{endpoint.scoring_uri}/v1/chat/completions"

payload = {
    "max_tokens": 1024,
    "messages": [{"content":"This script is great so far. Can you add more dialogue between Amanda and Thierry to build up their chemistry and connection?","role":"user"}]
}
headers = {
    "Content-Type": "application/json",
    "Authorization": "your-api-key"
}

response = requests.post(url, json=payload, headers=headers)

response.json()